In [1]:
import intake
import xarray as xr
import zarr
import numpy as np

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../src/')))

from weather_router import isochronal_weather_router, polar, point_validity, visualize


In [2]:

if not os.path.exists('cache.zarr/'):
    print('Downloading data...')
    ds = xr.open_zarr(
        'gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3',
        chunks=None,
        storage_options=dict(token='anon'),
    )
    ds = ds.rename({'latitude':'lat', 'longitude':'lon'})
    ds = ds[['10m_u_component_of_wind', '10m_v_component_of_wind']]
    ds.coords['lon'] = ((ds.coords['lon'] + 180) % 360) - 180
    ds = ds.sortby(ds.lon)
    ds = ds.sel(lat = slice(50,25)).sel(lon = slice(-20,10))
    ds = ds.sel(time = slice('2025-01-13T12:00:00', '2025-01-20T12:00:00'))
    ds = ds.load()
    u10 = ds['10m_u_component_of_wind']
    v10 = ds['10m_v_component_of_wind']
    tws = xr.ufuncs.hypot(v10, u10)
    tws = tws*1.94384 #convert m/s to knots
    twd = (180 + np.rad2deg(np.arctan2(u10, v10))) % 360
    ds = tws.to_dataset(name = 'tws')
    ds['twd'] = twd
    ds.to_zarr('cache.zarr')
else:
    ds = xr.open_zarr('cache.zarr')

In [3]:
def get_wind(t, lat, lon):
    tws_sel = ds.tws.sel(time = t, method = 'nearest')
    tws_sel = tws_sel.sel(lat = lat, lon = lon, method = 'nearest')
    twd_sel = ds.twd.sel(time = t, method = 'nearest')
    twd_sel = twd_sel.sel(lat = lat, lon = lon, method = 'nearest')
    return (np.float32(twd_sel.values), np.float32(tws_sel.values))

In [4]:
polar = polar.Polar('../test/volvo70.pol')

point_valid = point_validity.land_sea_mask(extent = [40,-7,35,4]).point_validity_arr

Palma = (39.283014, 2.527704)
Gibraltar = (36.073, -5.354)

weatherrouter = isochronal_weather_router.weather_router(polar, 
                                                         get_wind, 
                                                         time_steps = ds.time.values,
                                                         step = 1,
                                                         start_point = Palma,
                                                         end_point = Gibraltar,
                                                         point_validity = point_valid,
                                                         spread = 140,
                                                         wake_lim = 35,
                                                         rounding = 2)

In [5]:
weatherrouter.route()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
reached dest


In [6]:
route_df = weatherrouter.get_fastest_route()
print(route_df)

          lat       lon                time         twd        tws  \
0   39.283014  2.527704 2025-01-13 12:00:00   12.779800  14.141977   
1   39.046172  2.609407 2025-01-13 13:00:00   10.717545  15.628220   
2   38.843651  2.331024 2025-01-13 14:00:00   11.437820  17.758444   
3   38.622896  2.028554 2025-01-13 15:00:00   13.881592  18.595226   
4   38.431252  1.638365 2025-01-13 16:00:00   18.880676  19.403446   
5   38.251182  1.258048 2025-01-13 17:00:00   23.103180  17.569761   
6   38.084239  0.906171 2025-01-13 18:00:00   26.285034  17.068508   
7   37.963480  0.509356 2025-01-13 19:00:00   35.553329  14.605747   
8   37.861536  0.174540 2025-01-13 20:00:00   42.995834  14.498947   
9   37.811001 -0.151605 2025-01-13 21:00:00   49.512329  13.639548   
10  37.577794 -0.247199 2025-01-13 22:00:00   58.508865  15.072598   
11  37.300943 -0.380761 2025-01-13 23:00:00   56.617683  14.068272   
12  37.021978 -0.467867 2025-01-14 00:00:00   52.350723  13.963294   
13  36.770920 -0.563

In [7]:
viz = visualize.visualize(ds, Palma, Gibraltar, route_df, filename='route1')
viz.return_plot()

BokehModel(combine_events=True, render_bundle={'docs_json': {'a168c5c8-4190-43cf-be60-5bba41fd3f81': {'version…